# Part 1: Synthetic Data Generation and Training Workflow with Warehouse Sim Ready Assets

This notebook is the first part of the SDG and Training Workflow. We will be focusing on generating Synthetic Data for our use case

A high level overview of the steps:
* Pulling Isaac Sim Docker Container 
* Using Replicator API for Data Generation with Domain Randomization


### Table of Contents

This notebook shows provides an overview of generating synthetic data using Warehouse Sim Ready assets with Isaac Sim and Omniverse Replicator. We will generate data for the `palletjack` class of objects. 

1. [Set up Isaac Sim via Docker Container](#head-1)
2. [Generate Data for Detecting Palletjacks](#head-2)
3. [Deeper dive into SDG script](#head-3)


## 1. Set up Isaac Sim: Docker Container Installation <a class="anchor" id="head-1"></a>

### This step can be skipped if the Isaac Sim Docker container has already been set up on your Cloud/Remote Instance

* Follow the [instructions](https://docs.omniverse.nvidia.com/isaacsim/2022.2.1/install_container.html) for Isaac Sim Container Installation
* Ensure that `docker run` command on Step 7 works as expected and you are able to enter the container. 

We will use `./python.sh` in the container to run our SDG script. Please make sure you exit the container before running the next cells  

## 2. Generate Data for Detecting Palletjacks  <a class="anchor" id="head-2"></a>

* We can find the Palletjack USDs in the Warehouse Sim Ready asset collection (`http://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/DigitalTwin/Assets/Warehouse/Equipment/Pallet_Trucks`)
* First, we will mount our current local directory while running the docker container. This will ensure that we can run our scripts inside the Isaac Sim container. Data generated in the container will also be saved in this mounted directory.

In [1]:
import os

# This is the directory which will be mounted into the Isaac Sim container. Make sure <path_to_repo_cloned> is updated correctly
# os.environ["MOUNT_DIR"]=os.path.join(<path_where_repo_cloned>, "palletjack_sdg")
os.environ["LOCAL_PROJECT_DIR"]=os.path.dirname(os.getcwd())
os.environ["MOUNT_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR"), "palletjack_sdg")
print(os.getenv("MOUNT_DIR"))

/home/karma/Downloads/getting_started_v4.0.1/notebooks/tao_launcher_starter_kit/detectnet_v2/sdg_and_training/sdg-and-training/palletjack_sdg


In [ ]:
# Make sure the MOUNT_DIR location is correct, it shold have the scripts needed for SDG there

!docker run --name isaac-sim --entrypoint bash -it --gpus all -e "ACCEPT_EULA=Y" --rm --network=host \
    -v ~/docker/isaac-sim/cache/kit:/isaac-sim/kit/cache/Kit:rw \
    -v ~/docker/isaac-sim/cache/ov:/root/.cache/ov:rw \
    -v ~/docker/isaac-sim/cache/pip:/root/.cache/pip:rw \
    -v ~/docker/isaac-sim/cache/glcache:/root/.cache/nvidia/GLCache:rw \
    -v ~/docker/isaac-sim/cache/computecache:/root/.nv/ComputeCache:rw \
    -v ~/docker/isaac-sim/logs:/root/.nvidia-omniverse/logs:rw \
    -v ~/docker/isaac-sim/data:/root/.local/share/ov/data:rw \
    -v ~/docker/isaac-sim/documents:/root/Documents:rw \
    -v $MOUNT_DIR:/isaac-sim/palletjack_sdg \
    nvcr.io/nvidia/isaac-sim:2022.2.1 \
    ./palletjack_sdg/palletjack_datagen.sh
    
# Make sure $MOUNT_DIR is set correctly from the cell above


The data generation will begin in `headless` mode. We will be generating 5k images and using a 90:10 split for training and validation. 

In [ ]:
# Once the data generation is complete, list the folders in the data directory

!ls -rlt $MOUNT_DIR/palletjack_data

# There hould be 3 folders -> 1. distractors_warehouse 2. distractors_additional 3. no_distractors 

## 3. Deeper Dive into SDG Script  <a class="anchor" id="head-3"></a>

* The `standalone_palletjack_sdg.py` is the Python script which runs and generates data in headless mode inside the container.
* The overall flow of the script is similar to the `standalone_examples/replicator/offline_generation.py` file provided as a starting point with Isaac Sim


* We will be carrying out specific randomizations targeted to our use case. Some of them are:
    * Camera Pose Randomization -> Should be similar to a robot perspective in the scene
    * Palletjack Color Randomization -> To ensure model is robust to variations in Palletjack colors
    * Distractors Pose Randomization -> To enable the model to *focus* on the right object (Our object of interest: Palletjack)
    * Lighting Randomization-> Model robust to lights and reflections/shadows in the scene
    * Floor and Wall Texture Randomization -> Model more robust to changes in background textures and features <br> <br>
    
    
* We are only interested in the `palletjack` object class, all other semantics are removed from the stage with the `update_semantics()` function

* You can use a model of your own choice to train with this data (Pytorch/Tensorflow or other frameworks)

* The data is written in the KITTI Format, this allows seamless integration with TAO to train a model. Refer to `training/cloud_train.ipynb` notebook (Part 2) for training with TAO
